# BONUS-KAN MODEL

On the 30 of april 2024, it was published a paper (https://arxiv.org/abs/2404.19756) about a new possible approach towards neural networks. Kolmogorov-Arnold Networks (KANs) are proposed as an alternative to Multi-Layer Perceptrons (MLPs) inspired by the Kolmogorov-Arnold representation theorem. 

Unlike MLPs, which have fixed activation functions on nodes, KANs have learnable activation functions on edges. In simpler terms, KANs do not have linear weights, and instead, every weight parameter is replaced by a univariate function parameterized as a spline.

Since it is already been implemented on Python, we thought it would be interesting to test this model on our dataset, without any expectations of it being better than the MLP model, or to go deep into the theory behind it. 

In [ ]:
!pip install imodelsx

In [31]:
import imodelsx.kan
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch as th

In [36]:
# Load the dataset
df_seq_pathHFiltNorm = '../ai_lab/SmartSeq/HCC1806_SmartS_Filtered_Normalised_3000_Data_train.txt'
df_seq_HFiltNorm = pd.read_csv(df_seq_pathHFiltNorm, delimiter=' ')
df_seq_HfiltNormT = df_seq_HFiltNorm.T
#we now add the target variable
df_metaH_path = '../ai_lab/SmartSeq/HCC1806_SmartS_MetaData.tsv'
df_metaH = pd.read_csv(df_metaH_path, delimiter='\t')
df_metaH.set_index('Filename', inplace=True)

def add_target_column(data, metadata_path, target_column_name, target_category):
    df_meta = pd.read_csv(metadata_path, delimiter='\t')
    df_meta.set_index('Filename', inplace=True)
    merged_transition = pd.merge(data, df_meta[target_column_name], left_index=True, right_index=True)
    dummy_variables = pd.get_dummies(merged_transition[target_column_name], prefix='category').astype('int')
    
    # Set the target category to 1 and the others to 0
    dummy_variables['Target'] = dummy_variables[target_category]
    dummy_variables.drop(columns=[col for col in dummy_variables.columns if col != 'Target'], inplace=True)
    
    merged_data = pd.concat([merged_transition, dummy_variables], axis=1)
    merged_data.drop(columns=[target_column_name], inplace=True)
    return merged_data
merged = add_target_column(df_seq_HfiltNormT, df_metaH_path, 'Condition', 'category_Normo')

In [33]:
merged.head()

,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2,Target
output.STAR.PCRPlate1G12_Normoxia_S32_Aligned.sortedByCoord.out.bam,0,48,0,321,298,82,6250,634,0,0,...,10,136,0,0,29,0,29,0,0,1
output.STAR.PCRPlate1G1_Hypoxia_S102_Aligned.sortedByCoord.out.bam,8739,2101,55,96,1824,1938,62,0,522,413,...,0,264,0,134,68,0,213,0,0,0
output.STAR.PCRPlate1G2_Hypoxia_S2_Aligned.sortedByCoord.out.bam,13098,14032,0,0,1616,247,430,907,348,0,...,0,38,0,0,0,0,92,0,0,0
output.STAR.PCRPlate1G3_Hypoxia_S7_Aligned.sortedByCoord.out.bam,2880,356,0,6211,3,3430,79,1953,592,176,...,0,16,0,4,1,0,1,0,0,0
output.STAR.PCRPlate1G4_Hypoxia_S107_Aligned.sortedByCoord.out.bam,7777,5661,4383,0,145,4618,246,85,206,0,...,1,25,0,0,0,0,128,0,0,0


In [34]:
# Split X and y using ndarray
X = merged.iloc[:, :-1].values
y = merged.iloc[:, -1].values
print(X.shape, y.shape)
print(type(X), type(y))

(182, 3000) (182,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [35]:
# Model KAN
model = imodelsx.kan.KANClassifier(hidden_layer_size=8, device='cpu', regularize_activation=1.0, regularize_entropy=1.0)
model.fit(X, y)
#now we plot the process of training
y_pred = model.predict(X)

accuracy_score(y, y_pred)

  5%|▌         | 5/100 [00:00<00:04, 19.00it/s]

Early stopping


0.8076923076923077

As expected, the accuracy is lower than the other models, but it is interesting to see how the model behaves with our dataset.